# Analysis of the risk of bitcoin

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal

In [ ]:
# Get bitcoin price df and set date as index
df_btc = pd.read_csv("../data/BTC.csv", index_col="date", parse_dates=True)

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

In [ ]:
# Get OHLC average as price
df_btc["price"] = df_btc[["open", "high", "low", "close"]].mean(axis=1)
df_btc.drop(columns=["open", "high", "low", "close"], inplace=True)

## Volatility of bitcoin across time (EWMA method)

In [ ]:
# Get log price change
df_btc["price_change_log"] = np.log(df_btc["price"] / df_btc["price"].shift(1))

In [ ]:
# Biggest price changes
df_btc.loc[df_btc["price_change_log"].abs().sort_values(ascending=False).head(10).index, ["price_change_log"]].T

In [ ]:
# Get 90-day and 1-year volatility with the exponentially weighted moving average (EWMA) method
df_btc["volatility_90d"] = df_btc["price_change_log"].ewm(span=90).std()
df_btc["volatility_1y"] = df_btc["price_change_log"].ewm(span=365).std()

In [ ]:
# Change first cycle values to NaN to avoid having unreliable volatility measures
df_btc.loc[:df_btc.index[364], "volatility_1y"] = np.nan
df_btc.loc[:df_btc.index[89], "volatility_90d"] = np.nan

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_btc, x=df_btc.index, y="volatility_90d", label="90-day volatility", color="#00f8ff", linewidth=0.75)
sns.lineplot(data=df_btc, x=df_btc.index, y="volatility_1y", label="1-year volatility", color="#ff5b00", linewidth=1)

plt.title("Volatility of the price of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

save_chart_as_png("2_BTC_vlt")

In [ ]:
# All-time high 1-year volatility
ath_date = df_btc["volatility_1y"].idxmax()
df_btc.loc[[ath_date], ["price", "volatility_1y"]]

In [ ]:
# All-time low 1-year volatility
atl_date = df_btc["volatility_1y"].idxmin()
df_btc.loc[[atl_date], ["price", "volatility_1y"]]

In [ ]:
# Top 5 peaks of 1-year volatility
peaks, _ = signal.find_peaks(df_btc["volatility_1y"], distance=500)
df_btc.iloc[peaks].nlargest(5, "volatility_1y").sort_values("date")[["volatility_1y"]].T

In [ ]:
# Top 5 valleys of 1-year volatility
valleys, _ = signal.find_peaks(-df_btc["volatility_1y"], distance=500)
df_btc.iloc[valleys].nsmallest(5, "volatility_1y").sort_values("date")[["volatility_1y"]].T

In [ ]:
# Average 1-year volatility
df_btc["volatility_1y"].mean().round(3)

**Key takeaways:**
- ...

### Comparison of volatility with other assets

**Key takeaways:**
- ...

## Yearly volatility across time

In [ ]:
# Group by year and get standard deviation of price change along with the number of days
df_btc_yearly = df_btc.groupby(df_btc.index.year).agg(
    volatility=("price_change_log", "std"),
    num_days=("price_change_log", "count"),
)

In [ ]:
# Annualize the volatility for incomplete years (2010 and 2024), multiplying by the square root of the division of 365 by number of days
df_btc_yearly.loc[df_btc_yearly["num_days"] < 365, "volatility"] *= (365 / df_btc_yearly["num_days"])**0.5

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_btc_yearly, x=df_btc_yearly.index, y="volatility", color="#f7931a")

plt.title("Yearly volatility of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

save_chart_as_png("2_BTC_yearly_vlt")

In [ ]:
# Create table with yearly volatility stats
pd.DataFrame({
    "Average yearly volatility": [round(df_btc_yearly["volatility"].mean(), 4)],
    "Median yearly volatility": [round(df_btc_yearly["volatility"].median(), 4)],
    "Standard deviation": [round(df_btc_yearly["volatility"].std(), 4)],
    "Min yearly volatility": [round(df_btc_yearly["volatility"].min(), 4)],
    "Max yearly volatility": [round(df_btc_yearly["volatility"].max(), 4)],
})

**Key takeaways:**
- ...

## Volatility vs price